<a href="https://colab.research.google.com/github/vitorkawasaki/Alura_ImersaoIA/blob/main/teste__IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Final da Imersão em IA da Alura

* Nome: Vitor Kawasaki

## Entendendo o funcionamento das emoções de pessoas do Transtorno do Espectro Autista (TEA)

Com base um arquivo do Kaggle comparando com banco de dados da NDAR e fazendo perguntas referentes ao espectro e comparando os dados entre as duas pesquisas.

##Referências

* https://www.kaggle.com/datasets/divu2001/text-with-sentiment?resource=download
* National Database on Autism Research (NDAR)

##Instalando o SDK do Google

In [1]:
# Instalando o SDK do Google
!pip install -q -U google-generativeai

## Instalando o Accelerate

In [2]:
!pip install accelerate>=0.21.0

## Configurações iniciais

In [3]:
# Configurações iniciais
import google.generativeai as genai
from google.colab import userdata

## Carregamento CSV

In [4]:
# Carregar os dados do CSV em um DataFrame
import pandas as pd
df = pd.read_csv('/content/emotion_dataset_raw.csv')

## Pré-carregamento de dados

In [5]:
# Pré-processamento de dados, se necessário
# (limpeza de texto, remoção de caracteres especiais, etc.)

# Ajustar um modelo GPT pré-treinado
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

## Carregando modelo GPT

In [6]:
# Carregar o modelo GPT pré-treinado e o tokenizador
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Preparo dos dados

In [7]:
# Preparar os dados para treinamento
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/emotion_dataset_raw.csv',
    block_size=128
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


## Instalação Torch

In [8]:
!pip install transformers[torch]

## Definição de treinamento

In [9]:
# Definir os argumentos de treinamento
training_args = TrainingArguments(
    output_dir='./modelo_gpt_treinado',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=1_000,
    save_total_limit=1
)

##Treinando modelo

In [10]:
# Treinar o modelo
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)
trainer.train()

Step,Training Loss
500,4.195900
1000,4.029700
1500,3.943600
2000,3.893500
2500,3.865500
3000,3.863700
3500,3.822800
4000,3.792900
4500,3.757900
5000,3.776000


TrainOutput(global_step=7115, training_loss=3.8491246237523606, metrics={'train_runtime': 24093.8811, 'train_samples_per_second': 0.295, 'train_steps_per_second': 0.295, 'total_flos': 464773201920000.0, 'train_loss': 3.8491246237523606, 'epoch': 1.0})

## Adicionando API_KEY

In [11]:
# Carregar a chave de API do usuário
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

##Modelo generativo

In [35]:
# Criar um modelo generativo com as configurações de segurança
safety_settings = {
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
}
model = genai.GenerativeModel(
    model_name='gemini-1.5-pro-latest',
    generation_config=None,  # Você pode ajustar essa configuração conforme necessário
    safety_settings=safety_settings
)

## Configuração modelo generativo

In [37]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}
# primeiro é o parâmetro e depois é o nome utilizado, no caso foi usado o mesmo nome
# generation_config = nome_configuracao
model = genai.GenerativeModel(model_name = 'gemini-1.0-pro',
                                  generation_config = generation_config,
                                  safety_settings = safety_settings,)

In [39]:
# retorno recebe gera o conteúdo
response = model.generate_content("Me fale sobre quais emoções são ditas no banco de dados em CSV?")
print(response.text)

**Emoções comumente encontradas em bancos de dados CSV:**

* Raiva
* Tristeza
* Alegria
* Medo
* Surpresa
* Nojo
* Antecipação
* Confiança
* Desgosto
* Culpa
* Vergonha
* Alívio
* Felicidade
* Irritação
* Frustração
* Entusiasmo
* Motivação
* Ansiedade
* Estresse
* Paz
* Serenidade

**Como as emoções são representadas em CSV:**

As emoções podem ser representadas em CSV de várias maneiras, incluindo:

* **Valores textuais:** As emoções são escritas como palavras, como "raiva", "tristeza" ou "alegria".
* **Valores numéricos:** As emoções são atribuídas a números, como "1" para raiva, "2" para tristeza e assim por diante.
* **Valores codificados por cores:** As emoções são representadas por cores diferentes, como vermelho para raiva, azul para tristeza e amarelo para alegria.

**Considerações ao trabalhar com emoções em CSV:**

* **Codificação:** Certifique-se de que a codificação das emoções seja consistente em todo o banco de dados.
* **Interpretação:** As emoções podem ser subjetivas 

## Criando lista vazia

In [41]:
# [] lista vazia
chat = model.start_chat(history=[])

## Interação com perguntas e respostas

In [42]:
print("Digite 'fim' para finalizar o chat")
prompt = input('Esperando prompt: ')

while prompt != "fim":
  response = chat.send_message(prompt)
  print("Resposta:", response.text, '\n\n')
  print("Digite 'fim' para finalizar o chat")
  prompt = input('Esperando prompt: ')

Digite 'fim' para finalizar o chat
Esperando prompt: Me fale sobre o sentimento neutro do autismo
Resposta: **Sentimento Neutro do Autismo**

O sentimento neutro do autismo refere-se a um estado emocional em que os indivíduos autistas não experimentam emoções positivas ou negativas intensas. Pode ser caracterizado por:

* **Apatia:** Indiferença ou falta de interesse em atividades ou eventos.
* **Embotamento emocional:** Redução na intensidade das emoções, tanto positivas quanto negativas.
* **Dificuldade em expressar emoções:** Incapacidade de transmitir emoções de forma clara ou apropriada.
* **Falta de reações emocionais:** Ausência de respostas emocionais esperadas em situações sociais ou emocionais.

**Causas**

As causas do sentimento neutro do autismo são complexas e podem incluir:

* **Diferenças neurobiológicas:** Anormalidades nas regiões cerebrais responsáveis pelo processamento emocional.
* **Dificuldades sociais:** Interações sociais limitadas ou desafiadoras podem levar à

##Chat

In [43]:
chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.0-pro',
        generation_config={'candidate_count': 1, 'temperature': 0.5},
        safety_settings={<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>},
        tools=None,
        system_instruction=None,
    ),
    history=[glm.Content({'parts': [{'text': 'Me fale sobr...ro do autismo'}], 'role': 'user'}), glm.Content({'parts': [{'text': '**Sentimento...dade de vida.'}], 'role': 'model'}), glm.Content({'parts': [{'text': 'No teste em ...bre autistas?'}], 'role': 'user'}), glm.Content({'parts': [{'text': 'Sim, o conju...s envolvidos.'}], 'role': 'model'}), glm.Content({'parts': [{'text': 'poderia me p...los autistas?'}], '

In [44]:
chat.history

[parts {
   text: "Me fale sobre o sentimento neutro do autismo"
 }
 role: "user",
 parts {
   text: "**Sentimento Neutro do Autismo**\n\nO sentimento neutro do autismo refere-se a um estado emocional em que os indiv\303\255duos autistas n\303\243o experimentam emo\303\247\303\265es positivas ou negativas intensas. Pode ser caracterizado por:\n\n* **Apatia:** Indiferen\303\247a ou falta de interesse em atividades ou eventos.\n* **Embotamento emocional:** Redu\303\247\303\243o na intensidade das emo\303\247\303\265es, tanto positivas quanto negativas.\n* **Dificuldade em expressar emo\303\247\303\265es:** Incapacidade de transmitir emo\303\247\303\265es de forma clara ou apropriada.\n* **Falta de rea\303\247\303\265es emocionais:** Aus\303\252ncia de respostas emocionais esperadas em situa\303\247\303\265es sociais ou emocionais.\n\n**Causas**\n\nAs causas do sentimento neutro do autismo s\303\243o complexas e podem incluir:\n\n* **Diferen\303\247as neurobiol\303\263gicas:** Anormalidad

## Melhorando visualização do histórico

In [45]:
#Melhorando a visualização
#Código disponível em https://ai.google.dev/tutorials/python_quickstart#import_packages
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#Imprimindo o histórico
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
  print('-------------------------------------------')

> **user**: Me fale sobre o sentimento neutro do autismo

-------------------------------------------


> **model**: **Sentimento Neutro do Autismo**
> 
> O sentimento neutro do autismo refere-se a um estado emocional em que os indivíduos autistas não experimentam emoções positivas ou negativas intensas. Pode ser caracterizado por:
> 
> * **Apatia:** Indiferença ou falta de interesse em atividades ou eventos.
> * **Embotamento emocional:** Redução na intensidade das emoções, tanto positivas quanto negativas.
> * **Dificuldade em expressar emoções:** Incapacidade de transmitir emoções de forma clara ou apropriada.
> * **Falta de reações emocionais:** Ausência de respostas emocionais esperadas em situações sociais ou emocionais.
> 
> **Causas**
> 
> As causas do sentimento neutro do autismo são complexas e podem incluir:
> 
> * **Diferenças neurobiológicas:** Anormalidades nas regiões cerebrais responsáveis pelo processamento emocional.
> * **Dificuldades sociais:** Interações sociais limitadas ou desafiadoras podem levar à apatia e ao embotamento emocional.
> * **Experiências traumáticas:** Eventos traumáticos ou adversos podem suprimir as respostas emocionais.
> * **Condições comórbidas:** Transtornos como depressão ou ansiedade podem contribuir para o sentimento neutro.
> 
> **Implicações**
> 
> O sentimento neutro do autismo pode ter implicações significativas para os indivíduos afetados:
> 
> * **Dificuldades sociais:** A falta de reações emocionais pode dificultar a construção de relacionamentos e a comunicação.
> * **Problemas comportamentais:** A apatia e o embotamento emocional podem levar a problemas comportamentais, como isolamento ou retraimento.
> * **Baixa qualidade de vida:** A ausência de emoções positivas pode afetar negativamente a qualidade de vida e o bem-estar geral.
> 
> **Intervenções**
> 
> Existem várias intervenções que podem ajudar a abordar o sentimento neutro do autismo:
> 
> * **Terapia cognitivo-comportamental (TCC):** Ensina estratégias de enfrentamento para gerenciar emoções e melhorar as habilidades sociais.
> * **Terapia de aceitação e comprometimento (ACT):** Ajuda os indivíduos a aceitar suas emoções e se envolver em atividades valiosas.
> * **Terapia de arte ou música:** Fornece uma saída criativa para expressar emoções e melhorar a autoconsciência.
> * **Medicamentos:** Em alguns casos, medicamentos antidepressivos ou ansiolíticos podem ser prescritos para melhorar o humor e reduzir o embotamento emocional.
> 
> É importante observar que o sentimento neutro do autismo é um espectro, e os indivíduos podem experimentar diferentes graus de sintomas. Com intervenções adequadas, os indivíduos autistas podem aprender a gerenciar suas emoções e melhorar sua qualidade de vida.

-------------------------------------------


> **user**: No teste em CSV tem dados sobre autistas?

...

-------------------------------------------


> **user**: existem quantos tipos de sentimento neutro já documentado? se tiver mais de um, me explique cada um deles e o quanto pode impactar na vida social da pessoa 

-------------------------------------------


> **model**: Existem vários tipos de sentimento neutro documentados, cada um com suas características e impactos únicos na vida social:
> 
> **1. Sentimento Neutro Afetivo:**
> 
> * Caracterizado pela ausência de emoções positivas ou negativas intensas.
> * Os indivíduos podem se sentir calmos, equilibrados e sem emoções fortes.
> * Pode impactar a vida social ao dificultar a expressão e o reconhecimento de emoções nos outros.
> 
> **2. Sentimento Neutro Cognitivo:**
> 
> * Caracterizado pela falta de pensamentos ou avaliações emocionais.
> * Os indivíduos podem ter dificuldade em identificar e compreender suas próprias emoções e as dos outros.
> * Pode impactar a vida social ao prejudicar a capacidade de se conectar emocionalmente com os outros.
> 
> **3. Sentimento Neutro Motivacional:**
> 
> * Caracterizado pela falta de motivação ou desejo de se envolver em atividades.
> * Os indivíduos podem se sentir apáticos, desinteressados e sem energia.
> * Pode impactar a vida social ao dificultar a participação em atividades sociais e o estabelecimento de relacionamentos.
> 
> **4. Sentimento Neutro Emocional:**
> 
> * Caracterizado pela ausência de reações emocionais a estímulos que normalmente evocam emoções.
> * Os indivíduos podem parecer indiferentes ou desapegados emocionalmente.
> * Pode impactar a vida social ao prejudicar a capacidade de formar vínculos emocionais e responder adequadamente às situações sociais.
> 
> **Impactos na Vida Social:**
> 
> O sentimento neutro pode impactar significativamente a vida social de uma pessoa das seguintes maneiras:
> 
> * **Dificuldades na Comunicação Emocional:** Indivíduos com sentimento neutro podem ter dificuldade em expressar e interpretar emoções, o que pode dificultar a comunicação e a conexão com os outros.
> * **Isolamento Social:** A falta de reações emocionais e o desinteresse em atividades sociais podem levar ao isolamento social e à solidão.
> * **Mal-entendidos:** Os outros podem interpretar o sentimento neutro como indiferença ou falta de interesse, o que pode levar a mal-entendidos e conflitos.
> * **Dificuldades em Relacionamentos:** O sentimento neutro pode dificultar o estabelecimento e a manutenção de relacionamentos íntimos, pois pode prejudicar a capacidade de se conectar emocionalmente com os parceiros.

-------------------------------------------
